
# <font color=blue><div align="center">ST7 SNCF : Jalon 1</div></font>

## <font color=blue><div align="center">"<em>Indian Railways</em>"</div></font>

## Modules

In [44]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

In [45]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon1.xlsx"
test="mini_instance_test.xlsx"

## Traitement de données

In [46]:
def excel_to_dict(file :str)-> dict[str]:
    '''
    Takes excel file name in the repo
    and returns a dictionary[sheet_name]=Dataframe
    '''
    # Load the Excel file
    xl = pd.ExcelFile(file)
    
    # Get the names of the sheets
    sheet_names = xl.sheet_names
    
    # Initialize a dictionary to store the data
    data_dict = {}
    
    # Iterate over each sheet
    for sheet_name in sheet_names:
        # Read the sheet into a pandas DataFrame
        data_dict[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)
    return data_dict

def date_to_creneau(date0: datetime.datetime, hour: pd.DataFrame, date: pd.DataFrame):
    '''
    Takes hour and date's 1-column dataframes
    and returns a dataframe of 1-column of 'créneaux'
    starting from date0
    '''
    datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)))
    return (datetime_df-date0).dt.total_seconds().div(60*15).astype(int)+1

def creneau_to_date(date0: datetime.datetime, creneau : int):
    return date0+ datetime.timedelta(minutes=creneau*15)

# Instance sur laquelle on travaillera par la suite
INSTANCE= excel_to_dict(mini_instance)

### Dictionnaires utiles 

Trains de l'arrivée et du départs

In [47]:
trains_arrivee=[]

for index, row in INSTANCE["Sillons arrivee"].iterrows():
    numero_train = row['n°TRAIN']
    trains_arrivee.append(numero_train)

trains_depart=[]

for index, row in INSTANCE["Sillons depart"].iterrows():
    numero_train = row['n°TRAIN']
    trains_depart.append(numero_train)

nombre_trains=len(trains_arrivee)+len(trains_depart)
nombre_trains_arrivee=len(trains_arrivee)
nombre_trains_depart=len(trains_arrivee)

Correspondance

In [48]:
def trains_depart_relatifs_a(train_depart)->list:
    '''
    Prend l'id d'un train du départ 
    et retourne une liste des id des trains
    d'arrivée qui le constituent
    '''
    trains_arrivee_associes = []
    for index, row in INSTANCE["Correspondances"].iterrows():
        if row['n°Train depart']==train_depart:
            trains_arrivee_associes.append(row['n°Train arrivee'])
        
    return trains_arrivee_associes

création des créneaux

In [89]:
def numero_jour_semaine(date_str):
    # Convertir la chaîne de caractères de date en objet datetime en spécifiant le format
    date = pd.to_datetime(date_str, format='%d/%m/%Y')
    # Obtenir le numéro du jour de la semaine en utilisant la méthode dayofweek, ajouté de 1 pour avoir Lundi=1, Dimanche=7
    numero_jour = date.dayofweek
    
    return numero_jour

def HARR_to_creneau_in_h(heure):
    heure_str = heure.strftime("%H:%M")
    heure,minute= int(heure_str.split(':')[0]), int(heure_str.split(':')[1])
    
    return heure*4 + minute//15

In [95]:
!pip install datetime

     -------------------------------------- 43.1/43.1 kB 420.8 kB/s eta 0:00:00
   ---------------------------------------- 52.5/52.5 kB 299.8 kB/s eta 0:00:00
   -------------------------------------- 204.3/204.3 kB 496.4 kB/s eta 0:00:00


In [97]:
from datetime import datetime

In [98]:
datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

datetime.datetime(2005, 6, 1, 13, 33)

In [118]:
a=datetime.strptime('25-02-2002', '%d-%m-%Y').date()
b=datetime.strptime('26-02-2002', '%d-%m-%Y').date()
print(int((a-b).days))


-1


In [134]:
jour0=str(INSTANCE["Sillons arrivee"]["JARR"][0])
# print(jour0)
indice_jour0=numero_jour_semaine(jour0)
# print(indice_jour0)

def creneaux_arrivee(jour0,INSTANCE):
    creneaux_arrivee={}
    for index, row in INSTANCE["Sillons arrivee"].iterrows():
        delta=(datetime.strptime(row['JARR'],'%d/%m/%Y')-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_arrivee[row['n°TRAIN']]=(delta//7)*96+numero_jour_semaine(row['JARR'])*96 + HARR_to_creneau_in_h(row['HARR'])
    
    return creneaux_arrivee
    
def creneaux_depart(jour0,INSTANCE):
    creneaux_depart={}
    for index, row in INSTANCE["Sillons depart"].iterrows():
        # ti=datetime.fromtimestamp(row['JDEP']))
        # print(row['JDEP'].to_pydatetime())
        time_str=row['JDEP'].strftime('%Y-%m-%d')
        delta=(datetime.strptime(time_str,'%Y-%m-%d')-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_depart[row['n°TRAIN']]=(delta//7)*96*7+numero_jour_semaine(row['JDEP'])*96 + HARR_to_creneau_in_h(row['HDEP'])
    return creneaux_depart

print(creneaux_depart(jour0,INSTANCE))


{'sillon4': 180, 'sillon5': 180, 'sillon6': 182}


Créneaux de l'arrivée et du départ

In [49]:
# Date de référence et date finale 
date0=str(INSTANCE["Sillons arrivee"]["JARR"][0])+' 00:00:00'
date0=pd.to_datetime(date0)
#datef=pd.to_datetime(INSTANCE["Sillons depart"]["JDEP"][nombre_trains_depart-1])+datetime.timedelta(hours=24)
datef=datetime.datetime(2023,5,3,0,0,0)
nombre_creneaux=96*8


# Crenau du départ et d'arrivée
INSTANCE["Sillons arrivee"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons arrivee"]["HARR"], INSTANCE["Sillons arrivee"]["JARR"])
INSTANCE["Sillons depart"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons depart"]["HDEP"], INSTANCE["Sillons depart"]["JDEP"])

creneau_arrivee={} 
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    numero_train = row['n°TRAIN']
    creneau_arrivee[numero_train]=row["creneau"]

creneau_depart={}
for index, row in INSTANCE["Sillons depart"].iterrows():
    numero_train = row['n°TRAIN']
    creneau_depart[numero_train]=row["creneau"]

In [50]:
print(creneau_arrivee)

{'sillon1': 37, 'sillon2': 53, 'sillon3': 65}


In [72]:
print(creneau_depart)

{'sillon4': 8341, 'sillon5': 8341, 'sillon6': 8343}


In [135]:
creneau_arrivee=creneaux_arrivee(jour0,INSTANCE)
creneau_depart=creneaux_depart(jour0,INSTANCE)
nombre_creneaux=96*8

## Modèle mathématique 
On a modélisé notre problème en partant du fait que les machines ne sont disponibles que pour un seul train et dans un seul créneau. \
De ce fait, on introduit les variables *binaires* suivantes : \
\
    - $(deb_{ijc})$ : variable binaire indiquant si la machine de débranchement est utilisée \
    pour le sillion d'arrivée d'identifiant $i$ dans le jour $j$ au créneau $c$ \
    - $(for_{ijc})$ : variable binaire indiquant si la machine de formation est utilisée \
    pour le sillion du départ d'identifiant $i$ dans le jour $j$ au créneau $c$ \
    - $(deg_{ijc})$ : variable binaire indiquant si la machine de dégarage est utilisée \
    pour le sillion du départ d'identifiant $i$ dans le jour $j$ au créneau $c$

Les créneaux ont une durée de $15 min$ correspondant à la durée d'une tâche machine. Les valeurs possibles des créneaux dépendent de l'instance, \
soit alors $c \in \{1,2,..., c_{max}\} $ où $c=1$ correspond au premier créneau de 15 min à parfir du premier jour de travail à $0:00$ et \
$c_{max}$ le dernier créneau du dernier jours de travail à partir de $23:45$.

On note ainsi : 
$$A = \{\text{Les identifiants des trains d'arrivée}\}$$
$$D = \{\text{Les identifiants des trains du départ}\}$$
$$C= \{1,2,..., c_{max}\}$$

In [136]:
# IMPLEMNTATION DES VARIABLES 
# Implémentation Python
m= Model('Modèle')
deb = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
form = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
deg = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deg_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

## Contraintes
### 1. Unique débranchement, formation et dégarage
Chaque train d'arrivé subit un seul et unique débranchement, chaque train du départ subit un seul et unique débranchement et dégarage. 
$$ \forall i \in A \quad \sum_{c} deb_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} for_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} deg_{i,c} = 1 $$

In [137]:
uniqueDeb_train = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeb_train{i}') for i in trains_arrivee}
uniqueform_train = {i : m.addConstr(quicksum([form[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueFor_train{i}') for i in trains_depart}
uniqueDeg_train = {i : m.addConstr(quicksum([deg[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeg_train{i}') for i in trains_depart}

### 2. Une seule tâche machine par créneau  
Chaque machine est exploitée par au plus un seul train sur chaque créneau.
$$ \forall c \in C \quad \sum_{i \in A} deb_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} for_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} deg_{i,c} \leq 1 $$

In [138]:
uniqueDeb_creneau = {c : m.addConstr(quicksum([deb[(i, c)] for i in trains_arrivee]) <=1, name = f'UniqueDeb_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueFor_creneau = {c : m.addConstr(quicksum([form[(i, c)] for i in trains_depart]) <=1, name = f'UniqueFor_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueDeg_creneau = {c : m.addConstr(quicksum([deg[(i, c)] for i in trains_depart ]) <=1, name = f'UniqueDeg_creneau{c}') for c in range(1, nombre_creneaux + 1)}


### 3. Débranchement se fait après l'arrivée du train
Pour chaque train d'arrivée $i$, le débranchement ne peut pas avoir lieu dans les créneaux précedent celui d'arrivée $c^{a}_{i}$ après une heure de tâches humaines.
$$ \forall i \in A \quad \sum_{c \space \leq \space c^{a}_{i}+4} deb_{i,c} = 0$$ 

In [139]:
deb_apres_arrivee = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, creneau_arrivee[i] + 5)]) ==0, name = f'deb_apres_arrivee{i}') for i in trains_arrivee}


### 4. Formation d'un train de départ 
Pour chaque train du départ $t$, la formation ne peut avoir lieu qu'après débranchement de tous les trains ayant les wagons qui le constituent. \
On note : $$ \forall t \in D \quad A_t = \{\text{Les identifiants des trains d'arrivée ayant au moins un wagon du train t}\}$$
On peut exprimer la contrainte comme suit : 
$$ \forall t \in D, \space\forall c \in C, \space\forall i \in A_t \quad for_{t,c} \leq \sum_{c' \space \lt \space c} deb_{i,c'}$$ 

In [140]:
for_apres_deb = {(t,c,i) : m.addConstr(form[(t,c)]<=quicksum([deb[(i, c_p)] for c_p in range(1, c)]), name = f'for_apres_deb{t}{c}{i}') for t in trains_depart for c in range(1,nombre_creneaux+1) for i in trains_depart_relatifs_a(t)}


### 5. Dégarage après formation
Pour chaque train du départ, le dégarage ne peut avoir lieu qu'après la formation du train et $150$ minutes soit $10$ créneaux de tâches humaines.
$$ \forall t \in D, \space\forall c \in C \quad deg_{t,c} \leq \sum_{c' \space \lt \space c-10} for_{t,c'}$$ 

In [141]:
deg_apres_for = {(t,c) : m.addConstr(deg[(t,c)]<=quicksum([form[(t, c_p)] for c_p in range(1, c-9)]), name = f'deg_apres_for{t},{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}


### 6. Dégarage avant départ
Pour chaque train du départ $t$, le dégarage ne peut pas avoir lieu dans les créneaux suivants \
celui de $20$ minutes (de tâches humaines) avant le créneau du départ $c^{d}_{t}$.
$$ \forall t \in D \quad \sum_{c \space \geq \space c^{d}_{t}-2} deg_{t,c} = 0$$ 

In [142]:
deg_avant_depart = {t : m.addConstr(quicksum([deg[(t, c)] for c in range(creneau_depart[t]-2, nombre_creneaux+1)])==0, name = f'depart_apres_deg{t}') for t in trains_depart}


### 7. Disponibilités des machines
Il y a des créneaux où les machines sont indisponible, soient alors  $ \space C_{deb}, C_{for}, C_{deg} \space$  ces crénaux d'indisponibilités.
$$ \forall i \in A \quad \sum_{c \space \in \space C_{deb}} deb_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} for_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{deg}} deg_{i,c} = 0 $$

In [58]:

def parse_unavailabilities(unavailability_str):
    slots=[]
    if unavailability_str != '0':
            periods = unavailability_str.split(';')
            for period in periods:
                day, time_range = period.strip('()').split(',')
                day = int(day)
                start_hour, end_hour = time_range.split('-')
                start_hour = int(start_hour.split(':')[0])
                end_hour = int(end_hour.split(':')[0])
                if end_hour<=start_hour:
                    delta=96-(start_hour-end_hour)*4

            
                    slots.extend([(day-1)*24*4 + start_hour*4+ i+1 for i in range(delta)])
                else:
                    # slots.append([(day-1)*96+start_hour*4+creneau for creneau in range(0,)])
                    for hour in range(start_hour, end_hour):
                        slots.extend([(day-1)*24*4 + i+1 for i in range(hour*4, (hour+1)*4)])
    return slots

def creneaux_de_periode(indispo_str): #jour, plage_horaire
    creneaux = []
    if indispo_str!='0':
        periodes = indispo_str.split(';')
        for periode in periodes:
            jour, plage_horaire = periode.strip('()').split(',')
            jour = int(jour)
            plage_horaire = plage_horaire.strip('()').split('-')
            heure_debut, minute_debut = map(int, plage_horaire[0].split(':'))
            creneau_debut=minute_debut//15+1
            heure_fin, minute_fin = map(int, plage_horaire[1].split(':'))
            creneau_fin=minute_fin//15+1
            if (heure_debut,minute_debut)>=(heure_fin,minute_fin):
                for c in range((jour-1)*96+heure_debut*4+creneau_debut,jour*96):
                    creneaux.append(c)
                for c in range(jour*96,jour*96+heure_fin*4+creneau_fin):
                    creneaux.append(c)
            else:
                for c in range((jour-1)*96+heure_debut*4+creneau_debut,(jour-1)*96+heure_fin*4+creneau_fin):
                    creneaux.append(c) 
    return creneaux

# data1 = INSTANCE1['Machines']
data = INSTANCE['Machines']

df = pd.DataFrame(data)

df['Indisponibilites']=df['Indisponibilites'].astype(str)
# Apply the function to each row
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode)


DEB_INDIS = df.loc[df['Machine'] == 'DEB', 'Indisponibilites_TimeSlots'].values[0]
FOR_INDIS = df.loc[df['Machine'] == 'FOR', 'Indisponibilites_TimeSlots'].values[0]
DEG_INDIS = df.loc[df['Machine'] == 'DEG', 'Indisponibilites_TimeSlots'].values[0]



In [59]:
print(creneaux_de_periode('(2,13:00-14:00);(1, 13:00-13:00))'))
print(creneaux_de_periode('(1, 13:00-13:00);(2,13:00-14:00))'))

[149, 150, 151, 152, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152]


In [60]:
print(creneaux_de_periode('(6,13:00-13:00);(7, 13:00-13:00))'))
print(nombre_creneaux)
print(creneaux_de_periode('(7,00:00-00:00)'))
print(creneaux_de_periode('(7,00:00-00:00)'))

[533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724]
768
[577, 578, 579, 580, 581, 582, 583,

In [61]:
df.head(3)

,Machine,Type de tache,Duree,Indisponibilites,Indisponibilites_TimeSlots
0,DEB,Débranchement,15,0,[]
1,FOR,Formation,15,"(6,13:00-13:00);(7, 13:00-13:00)","[533, 534, 535, 536, 537, 538, 539, 540, 541, ..."
2,DEG,Dégarage,15,0,[]


In [62]:
print(FOR_INDIS)

[533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724]


In [63]:
print(trains_arrivee)
print(trains_depart)

['sillon1', 'sillon2', 'sillon3']
['sillon4', 'sillon5', 'sillon6']


In [64]:
print(FOR_INDIS)
print(nombre_creneaux)

[533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724]
768


In [143]:
indisponibilite_deb = {i : m.addConstr(quicksum([deb[(i, c)] for c in DEB_INDIS]) ==0, name = f'indisponibilité_deb{i}') for i in trains_arrivee }
indisponibilite_for = {i : m.addConstr(quicksum([form[(i, c)] for c in FOR_INDIS]) ==0, name = f'indisponibilité_for{i}') for i in trains_depart}
indisponibilite_deg = {i : m.addConstr(quicksum([deg[(i, c)] for c in DEG_INDIS]) ==0, name = f'indisponibilité_deg{i}') for i in trains_depart}


## Résolution

In [144]:
objfunction = 1
m.setObjective(objfunction , GRB.MINIMIZE)
m.params.outputflag = 1
m.update()

In [67]:
m.params.outputflag = 0

In [145]:
m.optimize()
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10008 rows, 6912 columns and 2948982 nonzeros
Model fingerprint: 0xdb289465
Variable types: 0 continuous, 6912 integer (6912 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10008 rows and 6912 columns
Presolve time: 2.65s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 3.48 seconds (1.30 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


In [146]:
for train in trains_arrivee:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if deb[(train,c)].x==1:
            print('deb de {} se fait au créneau : {}, l arrivee de ce train est au creneau {} '.format(train,c,creneau_arrivee[train]))
            print('-------------------------------------------------------')


for train in trains_depart:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if form[(train,c)].x==1:
            print('For de {} se fait au créneau: {}, le depart de ce train se fait à {}'.format(train,c,creneau_depart[train]))
            print('-------------------------------------------------------')
        if deg[(train,c)].x==1:
            print('deg de {} se fait au créneau: {}, le depart de ce train se fait à {} '.format(train,c,creneau_depart[train]))
            print('-------------------------------------------------------')
        


sillon1
--------------------------------------------
deb de sillon1 se fait au créneau : 150, l arrivee de ce train est au creneau 132 
-------------------------------------------------------
sillon2
--------------------------------------------
deb de sillon2 se fait au créneau : 153, l arrivee de ce train est au creneau 148 
-------------------------------------------------------
sillon3
--------------------------------------------
deb de sillon3 se fait au créneau : 165, l arrivee de ce train est au creneau 160 
-------------------------------------------------------
sillon4
--------------------------------------------
For de sillon4 se fait au créneau: 167, le depart de ce train se fait à 180
-------------------------------------------------------
deg de sillon4 se fait au créneau: 177, le depart de ce train se fait à 180 
-------------------------------------------------------
sillon5
--------------------------------------------
For de sillon5 se fait au créneau: 154, le depart de 

In [70]:
print('deb de {} se fait au créneau : {}'.format(2,7))

deb de 2 se fait au créneau : 7


# Questions 
- Horraires du départ non conforme avec les créneaux machines
- Créneaux et jours ? indices dépendants 
- Il ya un choix à faire lors de la correspondance entre tâche humaine et créneaux où \
entre tâche machine et créneaux.